In [20]:
import os
from bs4 import BeautifulSoup
import time
import sys
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import plotly.express as px


In [11]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

df1 = pd.DataFrame()
city = "forked-river-nj"
base_url = 'https://www.zillow.com/'+ city +'/sold/'  

for beds_count in [1, 2, 3, 4, 5]:
    for baths_count in [1.0, 1.5, 2.0, 3.0, 4.0]:
        #Loop through all 20 of zillows pages
        for count in range(1, 20):
            with requests.Session() as s:
                
                #sleep so that they think we're human
                time.sleep(np.random.randint(1,5))
                
                if count == 1:
                    page = ""
                else: 
                    page = str(count) + "_p/"

                bed_part =  str(beds_count) + "-_beds/"
                bath_part = str(baths_count) + "-_baths/"

                url = base_url + bed_part + bath_part + page
                r = s.get(url, headers=req_headers)
                
                #stop if  zillow redirects you to original page, this means you've reached the end of available pages
                print(r.url)
                if count != 1 and r.url == base_url:
                    print("Cycled through all pages")
                    break
                
                ## append to soup
                soup = BeautifulSoup(r.content, 'html.parser')   
                
                #all for loops are pulling the specified variable using beautiful soup and inserting into said variable
                for i in soup:
                    #skip if none
                    if i.name == None:
                        continue

                    #find all articles with class "list-card
                    cards = soup.find_all('article', class_='list-card')

                    #find what we want in the cards
                    for card in cards:
                        if card.name == None:
                            continue
                        
                        #get zillow id
                        zpid = card.attrs["id"]

                        #Get Data on the page
                        address = getattr(card.find(class_= 'list-card-addr'), 'text', None)
                        price = getattr(card.find(class_= 'list-card-price'), 'text', None)
                        sold_date = getattr(card.find("div", class_= 'list-card-img-overlay'), 'text', None)

                        if card.find("a", class_= 'list-card-link') != None:
                            url = card.find("a", class_= 'list-card-link')['href']

                        details = card.find(class_= 'list-card-details')

                        #parse the details
                        beds = ""
                        baths = ""
                        sqft = ""
                        status = ""
                        if details != None:
                            for detail in details:
                                if "bds" in detail.text:
                                    beds = detail.text
                                if "ba" in detail.text:
                                    baths = detail.text
                                if "sqft" in detail.text:
                                    sqft = detail.text
                                if "list-card-statusText" in detail.text:
                                    status = detail.text

                        #store in a dataframe
                        df1 = df1.append({'zpid': zpid, "sold_date": sold_date, 'address': address, 'price': price, 'url': url, 'beds': beds, 'baths': baths, 'sqft': sqft}, ignore_index=True)
                        


https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/2_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/3_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/4_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/5_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/6_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/7_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/8_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/9_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/10_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/11_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/12_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/13_p/
https://www.zillow.com/forked-river-nj/sold/1-_beds/1.0-_baths/14_p/
https://www.zillow.com/forked-river-nj/sold/1-_

In [30]:
#remove duplicates
df = df1.drop_duplicates(subset=['zpid'], keep='first')

#reset index
df = df.reset_index(drop=True)

### Clean the data
#Remove the dollar sign
df['price'] = df['price'].str.replace('$', '')
df['price'] = df['price'].str.replace(',', '')

#Remove anything thats a million and convert to int
for i in range(len(df)):
    x = df['price'][i]
    if x is None:
        continue
    if 'M' in x:
        x = x.replace('M', '')
        x = float(x) * 1000000
        x = int(x)
        df['price'][i] = str(x)

#multiply by 1,000,000
df['price'] = df['price']

#remove " ba" from baths
df['baths'] = df['baths'].str.replace(' ba', '')
df['baths'] = df['baths'].str.replace('--', '')

#remove " bds" from beds
df['beds'] = df['beds'].str.replace(' bds', '')
df['beds'] = df['beds'].str.replace('--', '')

#remove " sqft" from sqft
df['sqft'] = df['sqft'].str.replace(' sqft', '')
df['sqft'] = df['sqft'].str.replace(',', '')
df['sqft'] = df['sqft'].str.replace('--', '')

#Clean sold date
df['sold_date'] = df['sold_date'].str.replace('Sold ', '')

#Drop rows with "Open" anywhere in sold_date
df = df[df['sold_date'].str.contains('Open') == False]

#convert to datetime
df['sold_date'] = pd.to_datetime(df['sold_date'])

#Remove row if address is NaN
df = df.dropna(subset=['address'])

#parse address into city, state, zip
df['city'] = df['address'].str.split(',').str[1]
df['state'] = df['address'].str.split(',').str[2].str.split(' ').str[1]
df['zip'] = df['address'].str.split(',').str[2].str.split(' ').str[2]

#replace all blank data with NaN
df = df.replace('', np.nan)


In [31]:
##Save CSV with today's date
today = time.strftime("%Y%m%d")
df.to_csv(city+'_'+today+'.csv')

In [120]:
######Analyze the data

#Pull together all CSV
files = [f for f in os.listdir('.') if f.endswith('.csv')]
#Only keep files with city variable in them
files = [f for f in files if city in f]

#Combine all CSV files into one
df = pd.concat([pd.read_csv(f) for f in files])

#Remove duplicates
df = df.drop_duplicates()

#Remove rows with NaN
df = df.dropna()

#sold_date to datetime
df['sold_date'] = pd.to_datetime(df['sold_date'])

#Convert to int
df['price'] = df['price'].astype(int)
df['sqft'] = df['sqft'].astype(int)
df['beds'] = df['beds'].astype(int)
df['baths'] = df['baths'].astype(int)

#Remove rows with 0, 1 or 999 sqft 
df = df[df['sqft'] > 1]
df = df[df['sqft'] != 999]

#calculate the average price per sqft
df['price_sqft'] = df['price'] / df['sqft']

#create plotly scatter chart of avg price per sqft over time
fig = px.scatter(df, x='sold_date', y='price_sqft', color ='price',  
                color_continuous_scale=px.colors.sequential.Turbo, range_color=(100000, 1000000), title='Price Per Square Foot Over Time in '+city)
fig.update_traces(marker={'size': 10, 'opacity': 0.75})


#### Now group the data by week and create an EMA with an 8 week window
##floor date by week
df['week_sold'] = df['sold_date'] - pd.to_timedelta(df['sold_date'].dt.dayofweek, unit='d')

#Group data by week
df2 = df.groupby(['week_sold'], as_index=False)["price_sqft"].mean()

#add price ema
df2['price_ema'] = df2['price_sqft'].ewm(span=8).mean()

##Add trace to plotly fig
fig.add_traces(
    list(px.line(df2, x='week_sold', y='price_ema').select_traces())
    )
fig.update_traces(line_color='#303030', line_width=5)
fig.show()
